In [1]:
# Ignoring the warnings
import warnings
warnings.filterwarnings('ignore')

# Imorting required libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Importing require AI/ML Libraries
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier,GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier as xgb
from sklearn.preprocessing import StandardScaler


In [2]:
# Location of the datasets
train_data=r'C:\Users\DELL\train_depression.csv'
test_data=r'C:\Users\DELL\test_depression.csv'

In [3]:
# Locading the datasets in memory
df_train=pd.read_csv(train_data)

### EDA

In [4]:
# Checking the % of null entries
df_train.isnull().sum()/len(df_train)*100

id                                        0.000000
Name                                      0.000000
Gender                                    0.000000
Age                                       0.000000
City                                      0.000000
Working Professional or Student           0.000000
Profession                               26.034115
Academic Pressure                        80.172708
Work Pressure                            19.842217
CGPA                                     80.171997
Study Satisfaction                       80.172708
Job Satisfaction                         19.836532
Sleep Duration                            0.000000
Dietary Habits                            0.002843
Degree                                    0.001421
Have you ever had suicidal thoughts ?     0.000000
Work/Study Hours                          0.000000
Financial Stress                          0.002843
Family History of Mental Illness          0.000000
Depression                     